In [ ]:
import cv2

cap = cv2.VideoCapture('../853889-hd_1920_1080_25fps.mp4')

while True:
    ret, frame = cap.read()
    if not ret:
        break

    cv2.imshow("Frame", frame)

    key = cv2.waitKey(1)
    if key == 27:
        break
cap.release()
cv2.destroyAllWindows()
        